# 1. Importar librerias

In [38]:
from kagglehub import dataset_download
import pandas as pd

# 2. Esta operación descarga el dataset de Kaggle.
> Puede tardar unos minutos si el dataset no estaba descargado anteriormente.

> Archivos descargados: 8.3 GB aproximadamente (33 Millones de registros aproximadamente). 

In [2]:
path = dataset_download("dasgroup/rba-dataset")
print(path)

C:\Users\carlo\.cache\kagglehub\datasets\dasgroup\rba-dataset\versions\1


# 3. Leer el dataset
> Puede tardar unos minutos.

> Ajustar `nrows` si es necesario, tomar menos registros si el equipo no tiene suficientes recursos.

In [3]:
df = pd.read_csv(f"{path}/rba-dataset.csv", nrows=2000000)

In [4]:
df.head()

,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,0,2020-02-03 12:43:30.772,-4324475583306591935,NaN,10.0.65.171,NO,-,-,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,1,2020-02-03 12:43:43.549,-4324475583306591935,NaN,194.87.207.6,AU,-,-,60117,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2,2020-02-03 12:43:55.873,-3284137479262433373,NaN,81.167.144.58,NO,Vestland,Urangsvag,29695,Mozilla/5.0 (iPad; CPU OS 7_1 like Mac OS X) ...,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,3,2020-02-03 12:43:56.180,-4324475583306591935,NaN,170.39.78.152,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,4,2020-02-03 12:43:59.396,-4618854071942621186,NaN,10.0.0.47,US,Virginia,Ashburn,398986,Mozilla/5.0 (Linux; U; Android 2.2) Build/NMA...,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 16 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   index                     int64  
 1   Login Timestamp           object 
 2   User ID                   int64  
 3   Round-Trip Time [ms]      float64
 4   IP Address                object 
 5   Country                   object 
 6   Region                    object 
 7   City                      object 
 8   ASN                       int64  
 9   User Agent String         object 
 10  Browser Name and Version  object 
 11  OS Name and Version       object 
 12  Device Type               object 
 13  Login Successful          bool   
 14  Is Attack IP              bool   
 15  Is Account Takeover       bool   
dtypes: bool(3), float64(1), int64(3), object(9)
memory usage: 204.1+ MB


In [13]:
df.shape

(2000000, 16)

# 4. Limpiar el dataset

In [14]:
df.duplicated().sum()

np.int64(0)

In [15]:
df.isnull().sum().sort_values(ascending=False)

Round-Trip Time [ms]        1907001
Region                         1117
City                            880
Device Type                     104
User ID                           0
Login Timestamp                   0
Country                           0
index                             0
IP Address                        0
ASN                               0
Browser Name and Version          0
User Agent String                 0
OS Name and Version               0
Login Successful                  0
Is Attack IP                      0
Is Account Takeover               0
dtype: int64

In [16]:
category_cols = df.select_dtypes(include=["object"]).columns

In [17]:
category_cols

Index(['Login Timestamp', 'IP Address', 'Country', 'Region', 'City',
       'User Agent String', 'Browser Name and Version', 'OS Name and Version',
       'Device Type'],
      dtype='object')

In [18]:
for col in category_cols:
    if df[col].nunique() <= 10:
        print(f"{col}: {df[col].unique()}")

Device Type: ['mobile' 'desktop' 'tablet' 'bot' 'unknown' nan]


In [28]:
df_clean = df.drop(columns=['User ID', 'User Agent String', 'Browser Name and Version', 'OS Name and Version',
                            'Round-Trip Time [ms]', 'ASN', "Login Timestamp"])

In [29]:
df_clean.loc[df_clean['Region'].isnull(), 'Region'] = 'Unknown'
df_clean.loc[df_clean['City'].isnull(), 'City'] = 'Unknown'
df_clean.loc[df_clean['Device Type'].isnull(), 'Device Type'] = 'unknown'

In [30]:
df_clean.isnull().sum().sort_values(ascending=False)

index                  0
IP Address             0
Country                0
Region                 0
City                   0
Device Type            0
Login Successful       0
Is Attack IP           0
Is Account Takeover    0
dtype: int64

In [35]:
map_device = {'mobile': 1, 'desktop': 2, 'tablet': 3, 'bot': 4, 'unknown': 5}

df_clean['DeviceTypeNum'] = (df_clean['Device Type'].map(map_device).fillna(map_device['unknown']).astype('Int64'))

df_clean['DeviceTypeNum'].unique()

<IntegerArray>
[1, 2, 3, 4, 5]
Length: 5, dtype: Int64

In [36]:
df_clean.shape

(2000000, 10)

In [37]:
df_clean.head()

,index,IP Address,Country,Region,City,Device Type,Login Successful,Is Attack IP,Is Account Takeover,DeviceTypeNum
0,0,10.0.65.171,NO,-,-,mobile,False,False,False,1
1,1,194.87.207.6,AU,-,-,mobile,False,False,False,1
2,2,81.167.144.58,NO,Vestland,Urangsvag,mobile,True,False,False,1
3,3,170.39.78.152,US,-,-,mobile,False,False,False,1
4,4,10.0.0.47,US,Virginia,Ashburn,mobile,False,True,False,1


# 5. Análisis exploratorio de datos (EDA)